In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
data = pd.read_csv('./input/copper_returns_5d_final.csv')
data.describe()

,LMCADS03,LMCADY,DXY,SPX,BCOM,MXWD,XAU,XAG,LMCADY_acu_5d,LMCADY_std_5d
count,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000,5539.000000
mean,-0.000185,-0.000178,0.000007,-0.000252,0.000072,-0.000205,-0.000274,-0.000106,-0.000867,0.014218
std,0.016059,0.016464,0.004793,0.011794,0.010250,0.009880,0.010722,0.019587,0.034782,0.008721
min,-0.112019,-0.110645,-0.024921,-0.103782,-0.054910,-0.085172,-0.097378,-0.123485,-0.185825,0.000805
25%,-0.008649,-0.008880,-0.002635,-0.005469,-0.005554,-0.004878,-0.005978,-0.009770,-0.021102,0.008616
50%,0.000000,0.000000,0.000000,-0.000390,0.000000,-0.000643,-0.000497,-0.000771,-0.001844,0.012148
75%,0.007562,0.007760,0.002658,0.003949,0.005375,0.003807,0.004908,0.008074,0.017065,0.017520
max,0.109603,0.109134,0.027541,0.136158,0.066117,0.105134,0.099792,0.226116,0.259832,0.091981


In [3]:
features = data.drop(columns=['Date','LMCADY_std_5d','LMCADY_acu_5d'])
targets = data['LMCADY_acu_5d']
print(features.shape, targets.shape)

(5539, 8) (5539,)


In [4]:
# scaler = StandardScaler()
# features_normalized = scaler.fit_transform(features)
# print(features_normalized.shape)
# features = features_normalized
features

,LMCADS03,LMCADY,DXY,SPX,BCOM,MXWD,XAU,XAG
0,0.001241,0.001047,-0.000767,0.002808,0.006127,0.001182,-0.003199,0.009327
1,-0.000733,-0.000182,0.002015,0.003064,-0.005829,0.002399,-0.002942,-0.000409
2,0.009474,0.009322,-0.004022,0.001404,0.007491,0.002585,0.007338,0.002853
3,-0.002514,-0.002130,-0.005961,-0.003226,-0.001953,-0.006740,0.002320,0.033895
4,0.005432,0.005893,0.004159,-0.008826,0.003441,-0.006092,-0.013172,-0.026096
...,...,...,...,...,...,...,...,...
5534,-0.009774,-0.009724,0.006872,0.014287,-0.004085,0.014765,-0.018041,-0.010309
5535,0.004627,0.004489,-0.005850,0.006588,0.002701,0.008892,0.011519,0.020312
5536,-0.010746,-0.010210,0.004903,-0.021980,0.016248,-0.018378,0.000000,-0.001531
5537,-0.013035,-0.013388,0.004977,0.000484,-0.014186,-0.003775,-0.014804,-0.014724


In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)


In [6]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [7]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

In [9]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[rmse])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                11800     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 11,851
Trainable params: 11,851
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)


Epoch 1/50
139/139 [==============================] - 5s 6ms/step - loss: 0.0011 - rmse: 0.0324 - val_loss: 9.6807e-04 - val_rmse: 0.0307
Epoch 2/50
139/139 [==============================] - 1s 5ms/step - loss: 0.0010 - rmse: 0.0311 - val_loss: 9.3389e-04 - val_rmse: 0.0301
Epoch 3/50
139/139 [==============================] - 1s 4ms/step - loss: 0.0010 - rmse: 0.0311 - val_loss: 9.0110e-04 - val_rmse: 0.0295
Epoch 4/50
139/139 [==============================] - 0s 3ms/step - loss: 0.0010 - rmse: 0.0311 - val_loss: 9.0176e-04 - val_rmse: 0.0295
Epoch 5/50
139/139 [==============================] - 1s 4ms/step - loss: 9.9951e-04 - rmse: 0.0311 - val_loss: 9.1664e-04 - val_rmse: 0.0298
Epoch 6/50
139/139 [==============================] - 1s 4ms/step - loss: 0.0010 - rmse: 0.0312 - val_loss: 9.1801e-04 - val_rmse: 0.0298
Epoch 7/50
139/139 [==============================] - 1s 4ms/step - loss: 9.9644e-04 - rmse: 0.0311 - val_loss: 9.0665e-04 - val_rmse: 0.0296
Epoch 8/50
139/139 [======

In [11]:
model.evaluate(X_test, y_test)


35/35 [==============================] - 0s 5ms/step - loss: 9.3229e-04 - rmse: 0.0301


[0.0009322912083007395, 0.030073203146457672]